In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark
import pandas as pd
import os
import requests
from datetime import datetime
import pyarrow

In [ ]:
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['JAVA_HOME'] = '/usr/local/jdk8u222-b10'
os.environ['HADOOP_USER_NAME']='hive'
os.environ['PYSPARK_PYTHON'] ='/HDFS01/anaconda3/envs/main/bin/python'
conf = pyspark.SparkConf().setAll([
     ('spark.driver.maxResultSize', '0'),
     ('spark.driver.memory', '2g'),
     ('spark.sql.repl.eagerEval.enabled','true'),
     ('hive.strict.managed.tables','false'),
     ('hive.metastore.uris', 'thrift://nn01.bigdata:9083'),
     ('metastore.client.capability.check','false')
    ])
spark = SparkSession.builder \
        .master("local[*]") \
        .appName("myApp") \
        .config(conf=conf) \
        .enableHiveSupport() \
        .getOrCreate();

In [ ]:
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("workby", StringType(), True),
    StructField("customer", StringType(), True),
    StructField("site", StringType(), True),
    StructField("malltype", StringType(), True),
    StructField("worktype", StringType(), True),
    StructField("details", StringType(), True),
    StructField("onsite", StringType(), True),
    StructField("time", StringType(), True),
    StructField("travel", StringType(), True),
    StructField("engineer", StringType(), True),
    StructField("status", StringType(), True),
    StructField("day", StringType(), True),
    StructField("date_start", StringType(), True),
    StructField("date_end", StringType(), True),
    StructField("status_work", StringType(), True),
    StructField("end_job_detail", StringType(), True),
    StructField("file_location", StringType(), True),
    StructField("update_by", StringType(), True)
])

In [ ]:
df = spark.createDataFrame(pd.read_json(f"http://engineer.da.co.th/api_json.php?all"), schema=schema)

In [ ]:
df2 = df.repartition(5, "date_start")

In [ ]:
df.write \
    .mode("overwrite") \
	.partitionBy("date_start") \
    .saveAsTable("pyspark.engineer")

In [ ]:
spark.read.table("pyspark.test3")

In [ ]:
os.system("sudo -u hdfs hdfs dfs -ls /user/hive/warehouse/pyspark.db/engineer_partition/date_start=2023-01-27")

In [ ]:
x = spark.createDataFrame(pd.read_sql("select * from raw_data_j1.j1_rank_no",con=)).toPandas()

In [ ]:
data = spark.read.table('etl_j4.manpower_resource_account_no_province') \
            .filter("series =='1'")

In [ ]:
if spark.sql('show tables in etl_j4').filter("tableName == 'manpower_resource_account_no_province3'").count() > 0:
    data.write \
        .mode('append') \
        .saveAsTable('etl_j4.manpower_resource_account_no_province2')
else:
    data.write \
    .mode('overwrite') \
    .saveAsTable('etl_j4.manpower_resource_account_no_province2')

In [ ]:
check